In [1]:
!pip freeze


absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.6.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.7
cuda-python==12.2.1
cudf-cu12 @ https://pypi.nvidia.c

# Импортируем модули и библиотеки

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
%matplotlib inline
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import utils

#Делим наш датасет на обучающую и тестовую выборку

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

#Нормализация данных

In [5]:
x_train = x_train / 255
x_test = x_test / 255

#Создание модели нейронной сети

In [6]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

#Компиляция модели

In [7]:
model.compile(optimizer=tf.keras.optimizers.SGD(), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#Обучение модели

In [9]:
model.fit(x_train, y_train, epochs=10)


Epoch 1/10
1875/1875 [==============================] - 11s 5ms/step - loss: 0.7362 - accuracy: 0.7632
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5147 - accuracy: 0.8250
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4696 - accuracy: 0.8390
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4460 - accuracy: 0.8466
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4278 - accuracy: 0.8526
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4145 - accuracy: 0.8575
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4023 - accuracy: 0.8600
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3923 - accuracy: 0.8642
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3838 - accuracy: 0.8670
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.37

# Проверка точности предсказания

In [10]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test loss: ', test_loss)
print('Test accuracy: ', test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.4139 - accuracy: 0.8521
Test loss:  0.4139026999473572
Test accuracy:  0.8521000146865845


#Проверка

In [11]:
predictions = model.predict(x_test)

313/313 [==============================] - 1s 2ms/step


In [20]:
predictions[6]

array([2.66432893e-02, 7.07353582e-04, 1.88956708e-02, 1.76222145e-03,
       8.32593560e-01, 2.86988594e-04, 1.15811929e-01, 1.10565325e-05,
       3.28469556e-03, 3.25438305e-06], dtype=float32)

#Предсказанный ответ нейросети

In [18]:
np.argmax(predictions[6])

4

#Реальный ответ

In [19]:
y_test[6]

4

In [17]:
class_names[np.argmax(predictions[6])]

'Coat'